# Network System Capstone Homework 3

## Import modules

In [ ]:
from __future__ import annotations

import statistics

import matplotlib.pyplot as plt # type: ignore
import protocols
from setting import Setting

ALL_PROTOCOLS = (
    protocols.aloha,
    protocols.slotted_aloha,
    protocols.csma,
    protocols.csma_cd,
)

## Code Test

In [ ]:
def create_setting() -> Setting:
    return Setting(
        host_num=3,
        total_time=100,
        packet_num=4,
        max_colision_wait_time=20,
        p_resend=0.3,
        packet_size=3,
        link_delay=1,
        seed=109652039,
    )


for protocol in ALL_PROTOCOLS:
    success_rate, idle_rate, collision_rate = protocol(create_setting(), True)
    print(f"success_rate: {success_rate:.2f}")
    print(f"idle_rate: {idle_rate:.2f}")
    print(f"collision_rate: {collision_rate:.2f}")
    print()

## Questions

### Common Operations

In [ ]:
from typing import Iterable, Callable


def run_simulation(get_setting: Callable[[], Setting]):
    results: dict[str, tuple[float, float, float]] = {}
    
    for protocol in ALL_PROTOCOLS:
        simulation_result = [protocol(get_setting(), False) for _ in range(100)]
        
        success_rate = statistics.mean(r[0] for r in simulation_result)
        idle_rate = statistics.mean(r[1] for r in simulation_result)
        collision_rate = statistics.mean(r[2] for r in simulation_result)

        results[protocol.__name__] = (success_rate, idle_rate, collision_rate)

    return results


def plot_performance_chart(title: str, x: Iterable[int], x_label: str, results: dict[str, list[tuple[float, float, float]]], accurate_x: bool = False):
    for index, label in enumerate(("Success Rate", "Idle Rate", "Collision Rate")):
        plt.title(title) # type: ignore
        plt.xlabel(x_label)  # type: ignore
        plt.ylabel(label) # type: ignore
        plt.plot(x, [r[index] for r in results["aloha"]], marker="x", markersize=5) # type: ignore
        plt.plot(x, [r[index] for r in results["slotted_aloha"]], marker="s", markersize=5) # type: ignore
        plt.plot(x, [r[index] for r in results["csma"]], marker="^", markersize=5) # type: ignore
        plt.plot(x, [r[index] for r in results["csma_cd"]], marker="o", markersize=5) # type: ignore
        if accurate_x:
            plt.xticks(x) # type: ignore
        plt.legend(("ALOHA", "Slotted ALOHA", "CSMA", "CSMA/CD")) # type: ignore
        plt.show() # type: ignore

### Question 1

Apply the following settings in all methods and plot the results. (2%)
- host_num_list = [2,3,4,6]
- packet_num_list = [1200,800,600,400] (To ensure that the total number of packets remains constant.)
- Setting(host_num=h, packet_num=p, max_colision_wait_time=20, p_resend=0.3) for h,p in zip(host_num_list, packet_num_list)

In [ ]:
host_num_list = (2, 3, 4, 6)
packet_num_list = (1200, 800, 600, 400)
results: dict[str, list[tuple[float, float, float]]] = {}


def create_setting(h: int, p: int) -> Setting:
    return Setting(
        host_num=h,
        packet_num=p,
        max_colision_wait_time=20,
        p_resend=0.3,
        seed=20230404,
    )


for h, p in zip(host_num_list, packet_num_list):
    result = run_simulation(lambda: create_setting(h, p))
    for protocol in ALL_PROTOCOLS:
        results.setdefault(protocol.__name__, []).append(result[protocol.__name__])

plot_performance_chart(
    title="Influence of Host Number",
    x=host_num_list,
    x_label="Host Number",
    results=results,
    accurate_x=True,
)

### Question 2

Define two expressions, one for calculating `max_colision_wait_time` and another for calculating `p_resend`, which should both include a coefficient parameter c ≥ 1 and other parameters. Write down the expressions in your report and modify the `Setting` class accordingly. (The subsequent questions 3~8 will be based on this new setting.) (2%)

- `max_colision_wait_time = ? * c` (Hint: Two parameters)
- `p_resend = ? / c` (Hint: One parameter)

- `max_colision_wait_time = host_num * packet_size * coefficient`
- `p_resend = 1 / (host_num * coefficient)`

### Question 3

Redo the simulations from question 1 using the updated settings for all methods. Plot the results and describe the influence of using these expressions. (8%)

- host_num_list = [2,3,4,6]
- packet_num_list = [1200,800,600,400]
- Setting(host_num=h, packet_num=p, coefficient=1) for h,p in zip(host_num_list, packet_num_list)

In [ ]:
def create_setting(h: int, p: int) -> Setting:
    return Setting(
        host_num=h,
        packet_num=p,
        coefficient=1,
    )


host_num_list = (2, 3, 4, 6)
packet_num_list = (1200, 800, 600, 400)
results: dict[str, list[tuple[float, float, float]]] = {}

for h, p in zip(host_num_list, packet_num_list):
    result = run_simulation(lambda: create_setting(h, p))
    for protocol in ALL_PROTOCOLS:
        results.setdefault(protocol.__name__, []).append(result[protocol.__name__])

plot_performance_chart(
    title="Influence of Host Number",
    x=host_num_list,
    x_label="Host Number",
    results=results,
    accurate_x=True,
)

### Question 4

What’s the influence of “coefficient” in all methods. Apply the following settings, plot the results, and describe them. (8%)

- Setting(coefficient=c) for c in range(start=1, stop=31, step=1)

In [ ]:
def create_setting(c: int) -> Setting:
    return Setting(coefficient=c)


results: dict[str, list[tuple[float, float, float]]] = {}

for c in range(1, 31):
    result = run_simulation(lambda: create_setting(c))
    for protocol in ALL_PROTOCOLS:
        results.setdefault(protocol.__name__, []).append(result[protocol.__name__])


plot_performance_chart(
    title="Influence of Coefficient",
    x=range(1, 31),
    x_label="Coefficient",
    results=results,
)

### Question 5

What’s the influence of “packet_num” in all methods. Apply the following settings, plot the results, and describe them. (10%)
- Setting(packet_num=p) for p in range(start=100, stop=1050, step=50)

In [ ]:
def create_setting(p: int) -> Setting:
    return Setting(packet_num=p)


results: dict[str, list[tuple[float, float, float]]] = {}

for p in range(100, 1050, 50):
    result = run_simulation(lambda: create_setting(p))
    for protocol in ALL_PROTOCOLS:
        results.setdefault(protocol.__name__, []).append(result[protocol.__name__])


plot_performance_chart(
    title="Influence of Packet Number",
    x=range(100, 1050, 50),
    x_label="Packet Number",
    results=results,
)

### Question 6

What’s the influence of “host_num” in all methods. Apply the following settings, plot the results, and describe them. (10%)
- Setting(host_num=h) for h in range(start=1, stop=11, step=1)

In [ ]:
def create_setting(h: int) -> Setting:
    return Setting(host_num=h)


results: dict[str, list[tuple[float, float, float]]] = {}

for h in range(1, 11):
    result = run_simulation(lambda: create_setting(h))
    for protocol in ALL_PROTOCOLS:
        results.setdefault(protocol.__name__, []).append(result[protocol.__name__])


plot_performance_chart(
    title="Influence of Host Number",
    x=range(1, 11),
    x_label="Host Number",
    results=results,
)

### Question 7

What’s the influence of “packet_size” in all methods. Apply the following settings, plot the results, and describe them. (10%)
- Setting(packet_size=p) for p in range(start=1, stop=20, step=1)

In [ ]:
def create_setting(p: int) -> Setting:
    return Setting(packet_size=p)


results: dict[str, list[tuple[float, float, float]]] = {}

for p in range(1, 20):
    result = run_simulation(lambda: create_setting(p))
    for protocol in ALL_PROTOCOLS:
        results.setdefault(protocol.__name__, []).append(result[protocol.__name__])


plot_performance_chart(
    title="Influence of Packet Size",
    x=range(1, 20),
    x_label="Packet Size",
    results=results,
    accurate_x=True,
)

### Question 8

What’s the influence of “link_delay” in CSMA and CSMA/CD? Apply the following settings, plot the results, and describe them. (10%)
- link_delay= [0,1,2,3]
- packet_size_list= [7,5,3,1] # To ensure that the packet_time remains constant.
- Setting(link_delay=l, packet_siz=p) for l,p in zip(link_delay_list, packet_size_list)

In [ ]:
def create_setting(l: int, p: int) -> Setting:
    return Setting(
        link_delay=l,
        packet_size=p,
    )


link_delay_list = (0, 1, 2, 3)
packet_size_list = (7, 5, 3, 1)
results: dict[str, list[tuple[float, float, float]]] = {}

for l, p in zip(link_delay_list, packet_size_list):
    result: dict[str, tuple[float, float, float]] = {}
    
    for protocol in (protocols.csma, protocols.csma_cd):
        simulation_result = [protocol(create_setting(l, p), False) for _ in range(100)]
        
        success_rate = statistics.mean(r[0] for r in simulation_result)
        idle_rate = statistics.mean(r[1] for r in simulation_result)
        collision_rate = statistics.mean(r[2] for r in simulation_result)

        result[protocol.__name__] = (success_rate, idle_rate, collision_rate)

    for protocol in (protocols.csma, protocols.csma_cd):
        results.setdefault(protocol.__name__, []).append(result[protocol.__name__])

for index, label in enumerate(("Success Rate", "Idle Rate", "Collision Rate")):
    plt.title("Influence of Link Delay") # type: ignore
    plt.xlabel("Link Delay")  # type: ignore
    plt.ylabel(label) # type: ignore
    plt.plot(link_delay_list, [r[index] for r in results["csma"]], marker="^", markersize=5) # type: ignore
    plt.plot(link_delay_list, [r[index] for r in results["csma_cd"]], marker="o", markersize=5) # type: ignore
    plt.xticks(link_delay_list) # type: ignore
    plt.legend(("CSMA", "CSMA/CD")) # type: ignore
    plt.show() # type: ignore